# Import section

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import requests, re, json
import random
from tqdm import tqdm
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
from keplergl import KeplerGl
import geopandas as gpd
from yelpapi import YelpAPI

# Intro

After scraping the real Estate data I would like to collect some additional features for the modeling. For this i will use different sources.

# Yelp API client

As a first instance I will set up an api account for yelp to scrape several data

In [3]:
api_key = XXXX
client_id = XXXX
# key and client id have been deleted for the github version

In [4]:
yelp_api = YelpAPI(api_key)

Unfortunately the yelp api client is limited to 5.000 request per day and within one request the maximum result is 50. To maximize the outcome i will include some search criteria and drop the outliers at the end. 

For the criteria i will use the postcodes of London as location and also the price categories from yelp. 

For the postcodes i found a CSV provided by the london goverment under https://data.london.gov.uk/dataset/postcode-directory-for-london

In [3]:
df_postcodes = pd.read_csv("london_postcodes.csv")

The data includes a lot of information which we don't need so we will clean it first

In [7]:
df_postcodes.columns

Index(['pcd', 'pcd2', 'pcds', 'dointr', 'doterm', 'oscty', 'ced', 'oslaua',
       'osward', 'parish', 'usertype', 'oseast1m', 'osnrth1m', 'osgrdind',
       'oshlthau', 'nhser', 'ctry', 'rgn', 'streg', 'pcon', 'eer', 'teclec',
       'ttwa', 'pct', 'itl', 'statsward', 'oa01', 'casward', 'park', 'lsoa01',
       'msoa01', 'ur01ind', 'oac01', 'oa11', 'lsoa11', 'msoa11', 'wz11', 'ccg',
       'bua11', 'buasd11', 'ru11ind', 'oac11', 'lat', 'long', 'lep1', 'lep2',
       'pfa', 'imd', 'calncv', 'stp'],
      dtype='object')

In [8]:
df_postcodes = df_postcodes[['pcd', 'lat', 'long']]

In [9]:
df_postcodes.drop_duplicates(inplace = True)

In [10]:
london_postcodes = df_postcodes['pcd'].tolist()
len(london_postcodes)

326214

The more detailed postcode list is huge, I will save it for later but for our scraping we will clean the column more. I only want to have the first three digits. 

In [4]:
df_postcodes_short = df_postcodes["pcd"].apply(lambda x: x[:3])

In [5]:
df_postcodes_short.shape

(326214,)

As a lot of postcode have the same first three digits i will drop the duplicates. 

In [6]:
df_postcodes_short = df_postcodes_short.drop_duplicates()

In [7]:
df_postcodes_short.shape

(216,)

As the dataframe is very big and causing an issue for the push request on github i am saving the short version within the folder and keep the full file as backup on my computer 

In [8]:
df_postcodes_short.to_csv(f'df_postcodes.csv')

we will directly load it again to have a save point

In [9]:
df_postcodes_short = pd.read_csv("df_postcodes.csv", index_col = 0)

Finally i will create a list of the postcodes which will be used for the yelp api to iterate through the different locations

In [10]:
postcode_short = df_postcodes_short["pcd"].tolist()

In [22]:
len(postcode_short)

216

# Additional Features 

## Gyms

As a first additional feature I want to include the locations of gyms in London. I will use the Yelp api to extract the information about the gyms. 

**Normal search for "gym" in London**

In [118]:
response = yelp_api.search_query(term = "Gym",
                                 location = "London",
                                 limit = 50)

cols = list(response['businesses'][0].keys())
df_gym = pd.DataFrame(columns=cols)
for biz in response['businesses']:
    df_gym = df_gym.append(biz, ignore_index=True)

I did a first test run to evaluate if the api is working and to have a first look at the data. After this i will scrape all of the data for the different postcodes which i created above and also for different price categories. 

**Different using the different postcodes and price categories for the api**

In [119]:
for i in postcode_short:
    for price in range(1,5):
        response = yelp_api.search_query(term = "Gym",
                                 location = i,
                                 categories = "gyms, All",
                                 price = price, 
                                 limit = 50)
        
        for biz in response['businesses']:
            df_gym = df_gym.append(biz, ignore_index=True)

In [130]:
df_gym.reset_index(drop=True, inplace=True)

To avoid duplicates i will directly drop all of them based on the id of the business:

In [140]:
df_gym = df_gym.drop_duplicates('id')

I would need to extract the latitude and longitude from the coordinates column to make it usable for my analysis. 

In [6]:
#I will create two functions to extract the coordinates and transform them into floats
def extract_lat (i):
    try:
        lat = float(i.split(" ")[1].replace(",",""))
        return lat
    except:
        return np.nan
    
def extract_long (i):
    try:
        long = float(i.split(" ")[3].replace("}",""))
        return long
    except:
        return np.nan 

In [28]:
df_gym["gym_lat"] = df_gym["coordinates"].apply(extract_lat)
df_gym["gym_long"] = df_gym["coordinates"].apply(extract_long)

I will only need certain columns so i will exclude the not needed ones and also reduce tha size of the csv file.

In [31]:
df_gym = df_gym[["name", "review_count", "rating", "price", "gym_lat", "gym_long"]]

Last but not least i will save the csv for the next steps.

In [33]:
df_gym.to_csv(f'df_gym.csv')

## Supermarkets

Also here we will us the api from Yelp to collect the different locations of different grocery stores in London. The procedure is exactly the same as for the gyms. The only difference is that i will scrape for different supermarket names: Tesco, Marks and Spencer, Whole Foods and Sainsbury.

###  Tesco

In [164]:
response = yelp_api.search_query(term = "Tesco",
                                 location = "London",
                                 limit = 50,
                                 categories = "grocery")

cols = list(response['businesses'][0].keys())
df_tesco = pd.DataFrame(columns=cols)
for biz in response['businesses']:
    df_tesco = df_tesco.append(biz, ignore_index=True)

In [165]:
for i in tqdm(postcode_short):
    for price in range(1,5):
        try:
            response = yelp_api.search_query(term = "Tesco",
                                 location = str(i),
                                 price = price, 
                                 limit = 50,
                                 categories = "grocery")
        
            for biz in response['businesses']:
                df_tesco = df_tesco.append(biz, ignore_index=True)
        except: 
            pass

100%|█████████████████████████████████████████| 216/216 [10:37<00:00,  2.95s/it]


Drop of duplicates based on the business id:

In [167]:
df_tesco = df_tesco.drop_duplicates('id')

After reviewing the data i realized that yelp also provided me with other supermarkets. I will filter the dataframe by the name to only receive tesco markets

In [196]:
df_tesco = df_tesco[df_tesco["name"].str.contains("Tesco" or "tesco")]
df_tesco.reset_index(drop=True, inplace = True)

Saving the data for safety:

In [ ]:
df_tesco.to_csv(f'df_tesco.csv')

### Marks and Spencer (only supermarkets)

In [15]:
response = yelp_api.search_query(term = "marks & spencer",
                                 location = "London",
                                 limit = 50,
                                 categories = "grocery")

cols = list(response['businesses'][0].keys())
df_marks = pd.DataFrame(columns=cols)
for biz in response['businesses']:
    df_marks = df_marks.append(biz, ignore_index=True)

In [16]:
for i in tqdm(postcode_short):
    for price in range(1,5):
        try:
            response = yelp_api.search_query(term = "marks & spencer",
                                 location = str(i),
                                 price = price, 
                                 categories = "grocery",           
                                 limit = 50)
        
            for biz in response['businesses']:
                df_marks = df_marks.append(biz, ignore_index=True)
        except: 
            pass

100%|██████████████████████████████████████████████████████████████████████████| 216/216 [06:50<00:00,  1.90s/it]


In [17]:
# droping duplicates
df_marks = df_marks.drop_duplicates('id')

In [23]:
df_marks = df_marks[df_marks["name"].str.contains("Marks")]
df_marks.reset_index(drop=True, inplace = True)

Saving the data for safety:

In [25]:
df_marks.to_csv(f'df_marks.csv')

### Whole Foods

In [199]:
response = yelp_api.search_query(term = "Whole Foods Market",
                                 location = "London",
                                 limit = 50,
                                 categories = "grocery")

cols = list(response['businesses'][0].keys())
df_whole = pd.DataFrame(columns=cols)
for biz in response['businesses']:
    df_whole = df_whole.append(biz, ignore_index=True)

In [200]:
for i in tqdm(postcode_short):
    for price in range(1,5):
        try:
            response = yelp_api.search_query(term = "Whole Foods Market",
                                 location = str(i),
                                 price = price, 
                                 categories = "grocery",           
                                 limit = 50)
        
            for biz in response['businesses']:
                df_whole = df_whole.append(biz, ignore_index=True)
        except: 
            pass

100%|█████████████████████████████████████████| 216/216 [10:26<00:00,  2.90s/it]


In [217]:
# droping duplicates
df_whole = df_whole.drop_duplicates('id')

Droping lines which dont include whole foods in their name 

In [212]:
df_whole = df_whole[df_whole["name"].str.contains("Whole Foods" or "whole foods")]
df_whole.reset_index(drop=True, inplace = True)

In [230]:
df_whole.to_csv(f'df_whole.csv')

### Sainsbury’s

In [220]:
response = yelp_api.search_query(term = "Sainsbury's",
                                 location = "London",
                                 limit = 50,
                                 categories = "grocery")

cols = list(response['businesses'][0].keys())
df_sains = pd.DataFrame(columns=cols)
for biz in response['businesses']:
    df_sains = df_whole.append(biz, ignore_index=True)

In [221]:
for i in tqdm(postcode_short):
    for price in range(1,5):
        try:
            response = yelp_api.search_query(term = "Sainsbury's",
                                 location = str(i),
                                 price = price, 
                                 categories = "grocery",           
                                 limit = 50)
        
            for biz in response['businesses']:
                df_sains = df_sains.append(biz, ignore_index=True)
        except: 
            pass

100%|█████████████████████████████████████████| 216/216 [09:59<00:00,  2.78s/it]


In [223]:
# droping duplicates
df_sains = df_sains.drop_duplicates('id')

In [225]:
# reviewing name column and droping non sainsbury stores
df_sains = df_sains[df_sains["name"].str.contains("Sainsbury" or "sainsbury")]
df_sains.reset_index(drop=True, inplace = True)

In [229]:
df_sains.to_csv(f'df_sains.csv')

### Merging the Supermarket datasets

Above we scraped the information for several supermarkets. For Safety reason i saved the dataframes. Now i will load all of them and merge them into one dataframe for the supermarkets. 

In [64]:
df_tesco = pd.read_csv("df_tesco.csv", index_col = 0)
df_marks = pd.read_csv("df_marks.csv", index_col = 0)
df_whole = pd.read_csv("df_whole.csv", index_col = 0)
df_sains = pd.read_csv("df_sains.csv", index_col = 0)

In [65]:
df_grocery = pd.concat([df_tesco, df_marks, df_whole, df_sains], ignore_index=True)

In [66]:
df_grocery["grocery_lat"] = df_grocery["coordinates"].apply(extract_lat)
df_grocery["grocery_long"] = df_grocery["coordinates"].apply(extract_long)

For our further analysis we will only need the columns ["name", "review_count", "rating", "price", "grocery_lat", "grocery_long"]

In [67]:
df_grocery = df_grocery[["name", "review_count", "rating", "price", "grocery_lat", "grocery_long"]]

**Adjusting the column with the names of the stores**

In [68]:
df_grocery["name"].value_counts().head(10)

Sainsbury's                100
Tesco Express               71
Tesco                       65
Tesco Stores                36
Marks and Spencer           28
Whole Foods Market          25
Sainsburys                  21
Tesco Superstore            18
Tesco Metro                 16
Sainsburys Supermarkets     10
Name: name, dtype: int64

I would like to unify the name column more. It should only include "Sainsbury", "Tesco", "Marks and Spencer" and "Whole Foods".

In [69]:
def adj_names_supermarkets(i):
    if "tesco" in i.lower():
        return "Tesco"
    elif "sainsb" in i.lower():
        return "Sainsbury"
    elif "marks" in i.lower():
        return "Marks and Spencer"
    elif "whole" in i.lower():
        return "Whole Foods"
    else:
        return "NAN"

In [70]:
df_grocery["name"] = df_grocery["name"].apply(adj_names_supermarkets)

In [71]:
df_grocery["name"].value_counts()

Tesco                224
Sainsbury            146
Marks and Spencer     40
Whole Foods           30
Name: name, dtype: int64

In [73]:
df_grocery.to_csv(f'df_grocery.csv')

## Public transport stations

The Zoopla website provided already some information about the close by public transport positions. But i would like to have the ability to calculate the distance on my own. For this reason i will include a DataFrame with information about tube and train station in London. 

I found the website https://www.doogal.co.uk/london_stations.php which collected already all London tube and train stations with their coordinates. The data is provided as CSV.

In [12]:
df_tube = pd.read_csv("Londonstations.csv")

In [13]:
df_tube

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508757,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516886,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN
...,...,...,...,...,...,...,...
648,Woodside Park,525725,192564,51.617868,-0.185426,4,N12 8SE
649,Woolwich,543931,178994,51.491578,0.071819,4,SE18 6EU
650,Woolwich Arsenal,543754,178803,51.489907,0.069194,4,SE18 6HX
651,Woolwich Dockyard,542738,178908,51.491108,0.054612,3,SE18 5JY
